In [ ]:
#switched to elu, with generator on 3 cameras

In [ ]:
import numpy as np
import pandas as pd
import cv2
import math
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Lambda, Flatten
from keras.layers.convolutional import Conv2D, Cropping2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras import backend as K
from keras.optimizers import SGD
from tqdm import tqdm
import random
from sklearn.utils import shuffle
import sys
import pprint

#print(K.image_data_format())
#'channels_first'
#K.set_image_data_format('channels_last')
#print(K.image_data_format())



In [ ]:
split_fraction = 25000

all_data = pd.read_csv("mydata2/driving_log.csv", names = ["center", "left", "right", "Steering", "throtle", "something","speed"])
all_data_shuffled = shuffle(all_data)

print (all_data.shape)

training_data = all_data_shuffled[:split_fraction]
validation_data = all_data_shuffled[split_fraction:]

print (training_data.shape)
print (validation_data.shape)



In [ ]:
#generator
def driving_generator(working_data,id):
    print ("\n generator (re)started - ",id, "\n")
    # creating generator that gives always balanced batches
    print ("working data shape", working_data.shape)
    pandas_batch = pd.DataFrame()
    images_list = []
    new_list = shuffle(working_data)
    
    max_steering = new_list.iloc[:,[3]].max()
    max_steering = max_steering.as_matrix()[0]

    min_steering = new_list.iloc[:,[3]].min()
    min_steering = min_steering.as_matrix()[0]
    
    buckets = 32
    counter = 0
    
    step = abs((min_steering - max_steering) / buckets)
    print("max", max_steering)
    print("min", min_steering)
    print ("step", step)
    
    steering_range_low = min_steering
    steering_range_high = 0
    pp = pprint.PrettyPrinter(indent=4)
    
    while True:
        
        print("max", max_steering)
        print("min", min_steering)
        print ("step", step, "\n\n")

        counter += 1
        rest_of_counter = counter % 3
        print ("counter", counter, "Rest", rest_of_counter,"\n")
        steering_range_low = min_steering
        steering_range_high = 0
        i=0;
        for i in range(buckets):

                steering_range_high = steering_range_low + step
                
                bucket_list = new_list[(new_list.Steering >= steering_range_low) & (new_list.Steering < steering_range_high)]
                bucket_quantity = len(bucket_list)
                print ("high %.5f" % steering_range_high,"low %.5f" % steering_range_low, "i",i ,"bucket_quantity", bucket_quantity, "\n")
                
                if bucket_quantity >0:
                    chosen = bucket_list.sample(1, replace=True)
                    #print (chosen)
                    pandas_batch = pandas_batch.append(chosen)
                steering_range_low += step
        print ("selected rows", pandas_batch.shape)
        if rest_of_counter == 0:
            # center
            print ("center batch")
            names = pandas_batch.iloc[:,[0]].as_matrix()[:,0]
            steering = pandas_batch.iloc[:,[3]].as_matrix()[:,0]
        elif rest_of_counter ==1:
            print ("left batch")
            # left
            names = pandas_batch.iloc[:,[1]].as_matrix()[:,0]
            steering = pandas_batch.iloc[:,[3]].as_matrix()[:,0]+0.2
        else:
            # right
            print ("right batch")
            names = pandas_batch.iloc[:,[2]].as_matrix()[:,0]
            steering = pandas_batch.iloc[:,[3]].as_matrix()[:,0]-0.2
        #print ("\n names\n",names, "\n\n")
        #print ("steering", steering)
        selected_file_names = []
        
        for name in names:
            #print (name)
            #print (name.split('\\'))
            if name.startswith('/'):
                #print (name[0].split('/'))
                splited_name = name.split('/')
                path = 'mydata2/IMG/'+splited_name[9]
                selected_file_names.append(splited_name[9])
                                
                #print ("media",path)
            else:
                #print (name.split('\\'))
                splited_name = name.split('\\')
                path = 'mydata2/IMG/'+splited_name[6]
                selected_file_names.append(splited_name[6])
                #print (path)
            
            
            image = cv2.imread(path)
            image = image[...,::-1] #conversion to rgb
            #plt.imshow(image)
            #plt.show()
            #print(image)
            images_list.append(image)
        pp.pprint(selected_file_names)
        
        X_train = np.array(images_list)
        y_train = np.expand_dims(steering, axis=1)

        pandas_batch = pd.DataFrame()
        images_list = []
        #print ("xtrain", X_train,"ytrain", y_train)
        print ("\n\n ****************************************** end of batch *****************\n\n")
        # unpack batch properly
        yield (X_train, y_train)

#franek = driving_generator(training_data)
#next(franek)
#next(franek)
#raise SystemExit()


In [ ]:



def nvidia_model():
    model = Sequential()
    nv_model = Sequential()
    nv_model.add (Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
    nv_model.add (Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
    
    nv_model.add (Conv2D(24, (5,5), strides=1, padding="valid"))
    nv_model.add (ELU())
    nv_model.add ((MaxPooling2D(pool_size=(2,2))))

    nv_model.add (Conv2D(36, (5,5), strides=1, padding="valid"))
    nv_model.add (ELU())
    nv_model.add ((MaxPooling2D(pool_size=(2,2))))
        
    nv_model.add (Conv2D(48, (5,5), strides=1, padding="valid"))
    nv_model.add (ELU())
    nv_model.add ((MaxPooling2D(pool_size=(2,2))))
    
    nv_model.add (Conv2D(64, (3,3), strides=1, padding="valid"))
    nv_model.add (ELU())
    
    nv_model.add (Conv2D(64, (3,3), strides=1, padding="valid"))
    nv_model.add (ELU())
    
     
    nv_model.add(Flatten())
    nv_model.add(Dense(1164))
    nv_model.add(ELU())
    nv_model.add(Dense(100))
    nv_model.add(ELU())
    nv_model.add(Dense(50))
    nv_model.add(ELU())
    nv_model.add(Dense(10))
    nv_model.add(ELU())
    
    nv_model.add(Dense(1, activation="linear"))
    nv_model.compile(optimizer="adam", metrics=['accuracy'], loss="mse")
    return nv_model

my_model = nvidia_model()


training_generator = driving_generator(training_data, "training")
validation_generator = driving_generator(validation_data, "validation")

#my_model.fit(X_train, y_train,  validation_data=(X_valid,y_valid), shuffle=True, batch_size=32, epochs=5, verbose=1)
#model.fit_generator(train_generator, samples_per_epoch= /
          #  len(train_samples), validation_data=validation_generator, /
          #  nb_val_samples=len(validation_samples), nb_epoch=3)
        
my_model.fit_generator(training_generator,  validation_data=validation_generator, validation_steps=(validation_data.shape[0] // 32), steps_per_epoch=1000, epochs=7, verbose=1)
my_model.save("model.h5")
#score = model.evaluate(x_test, y_test, batch_size=32)


